# 1. Laurie's Books Customer Segmentation Analysis Preprocessing
Here we analyze and clean the raw transactions data.

In [1]:
import numpy as np
import pandas as pd

In [2]:
sales = pd.read_csv('20240301SalesDetail.csv')

In [3]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6117 entries, 0 to 6116
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           6117 non-null   object 
 1   Time           6117 non-null   object 
 2   Age            6117 non-null   object 
 3   ISBN           6117 non-null   object 
 4   Title          6117 non-null   object 
 5   Authors        5755 non-null   object 
 6   Line Type      6117 non-null   object 
 7   Sale/Order ID  6117 non-null   int64  
 8   Condition      6101 non-null   object 
 9   Section        6092 non-null   object 
 10  Location       6101 non-null   object 
 11  Binding        6019 non-null   object 
 12  Publisher      5844 non-null   object 
 13  Source         3026 non-null   object 
 14  Event          0 non-null      float64
 15  Quantity       6117 non-null   int64  
 16  Unit           6117 non-null   object 
 17  Credit Used    6117 non-null   object 
 18  Discount

We see that many columns are encoded as strings including the date. PII customer names are in the `Customer` column. The price of the item is in `Unit`. There are no unique customer names but the unique order ID is in the `Sale/Order ID` column. 

### Check values of various columns

In [4]:
sales.Customer.value_counts().head(1)

Customer
no  account    198
Name: count, dtype: int64

It looks like we need to clean out "no  account" and other non-name values.

In [5]:
sales.describe()

,Sale/Order ID,Event,Quantity,Store Rem.,Chain Rem.,On Order,Barcode
count,6117.000000,0.0,6117.000000,6117.000000,6117.000000,6117.0,5.701000e+03
mean,11004.231976,NaN,1.027628,-1.775707,-1.775707,0.0,9.789910e+12
std,547.554823,NaN,0.494630,17.885391,17.885391,0.0,4.972033e+04
min,10005.000000,NaN,-1.000000,-121.000000,-121.000000,0.0,9.789910e+12
25%,10546.000000,NaN,1.000000,-1.000000,-1.000000,0.0,9.789910e+12
50%,11012.000000,NaN,1.000000,0.000000,0.000000,0.0,9.789910e+12
75%,11472.000000,NaN,1.000000,1.000000,1.000000,0.0,9.789910e+12
max,11938.000000,NaN,28.000000,93.000000,93.000000,0.0,9.789910e+12


We need to look at that 28 quantity order as well as other quantities > 1. After doing that, it appears that we only need to remove orders from Section "Bookmarks". 

In [6]:
# Make a dataframe of unique names dropping na values.
customers_df = pd.DataFrame(sales.Customer.dropna().unique(), columns=['name'])

In [7]:
len(customers_df)

891

In [8]:
# Remove names that have 'abc' or 'account' in them. 
customers = customers_df[~customers_df.name.str.contains('abc|account')]

In [9]:
len(customers)

889

### Anonymize the names and save them as a separate key file

In [10]:
from faker import Faker
fake = Faker()
# The ean function can be set to 8 or 16 characters. The unique function ensures we avoid duplicates.
uid = [int(fake.unique.ean(length=8)) for i in range(len(customers))]

In [11]:
uid[:5]

[90591877, 72569436, 86516143, 92523166, 13995799]

In [12]:
customers_key = customers.assign(uid=uid)

In [13]:
customers_key.to_csv('customers_key.csv', index=False)

### Clean the sales data
Drop NAs and add uids. Keep genres and authors for future product mix analysis.

In [14]:
# Add the uids to the rest of the data
transactions_raw = pd.merge(sales, customers_key, how='inner', left_on='Customer', right_on='name')

In [15]:
# Create a base transaction DataFrame for cleaning
t_base = transactions_raw[['uid', 'Sale/Order ID', 'Date', 'Quantity', 'Unit', 'Section', 'Authors']]

In [16]:
t_base.head()

,uid,Sale/Order ID,Date,Quantity,Unit,Section,Authors
0,90591877,10005,2023-09-22,1,$5.00,American History,"Sandford, John"
1,90591877,10005,2023-09-22,1,$5.00,American History,"Sandford, John"
2,90591877,10005,2023-09-22,1,$5.00,American History,"Sandford, John"
3,90591877,10005,2023-09-22,1,$5.00,American History,"Sandford, John"
4,90591877,10005,2023-09-22,1,$5.00,American History,"Sandford, John"


In [17]:
t_renamed = t_base.rename(columns= {'Sale/Order ID': 'order_no', 
                                    'Date':'date',
                                    'Quantity':'qty',
                                    'Unit':'price',
                                    'Section':'genre',
                                    'Authors':'author'})

In [18]:
t_renamed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5708 entries, 0 to 5707
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   uid       5708 non-null   int64 
 1   order_no  5708 non-null   int64 
 2   date      5708 non-null   object
 3   qty       5708 non-null   int64 
 4   price     5708 non-null   object
 5   genre     5689 non-null   object
 6   author    5402 non-null   object
dtypes: int64(3), object(4)
memory usage: 312.3+ KB


In [19]:
# Convert the date to a date data type
t_renamed.date = pd.to_datetime(t_renamed.date)

In [20]:
# Convert the price by first stripping out the non-number characters
t_renamed.price = t_renamed.price.map(lambda x: float(x.replace('$', '').replace('(', '').replace(')', '')))

In [21]:
t_renamed.to_csv('transactions_cleaned.csv', index=False)